# Task 1: Text Classification

1. Download the BBC dataset provided on Moodle. The dataset, created by Greene and Cunningham, 2006 is a collection of 2225 documents from the BBC news website already categorized into 5 classes: business, entertainment, politics, sport, and tech.

In [1]:
import os

# The dataset can be found in the /data/BBC folder
print(os.listdir("../data/BBC"))

['business', 'entertainment', 'politics', 'README.TXT', 'sport', 'tech']


In [2]:
# Dictionary to hold the number of instances of each class
# business/entertainment/politics/sport/tech
category_dict = dict()

category_dict["business"] = len(os.listdir("../data/BBC/business"))
category_dict["entertainment"] = len(os.listdir("../data/BBC/entertainment"))
category_dict["politics"] = len(os.listdir("../data/BBC/politics"))
category_dict["sport"] = len(os.listdir("../data/BBC/sport"))
category_dict["tech"] = len(os.listdir("../data/BBC/tech"))

print("The number of instances in each class", count_dict)

NameError: name 'count_dict' is not defined

2. Plot the distribution of the instances in each class and save the graphic in a file called BBC-distribution.pdf. You may want to use matplotlib.pyplot and savefig to do this. This pre-analysis of the data set will allow you to determine if the classes are balanced, and which metric is more appropriate to use to evaluate the performance of your classifier.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
 
categories = list(category_dict.keys())
instances = list(category_dict.values())
  
fig = plt.figure(figsize = (10, 5))
 
# Creating the bar plot
plt.bar(categories, instances, color ='powderblue', width = 0.6)
 
plt.xlabel("Text Categories")
plt.ylabel("No. of Instances")
plt.title("Distribution of Instances in Each Class")
plt.savefig("../output/BBC-distribution.pdf")
plt.show()